<a href="https://colab.research.google.com/github/sunnyday2/flight_on_time/blob/desarrollo/flightontime_hackaton_e32_alura_latam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kaggle Notebook para un pre-procesamiento pesado**

---

Dado a que Colab no tiene suficientes recursos para elaborar el dataset a base de millones registros, usa Kaggle para:
* Lectura de datasets grandes (Parquet, CSV > varios GB)
* Limpieza inicial
* Feature engineering
* Agregaciones
* Generación de datasets intermedios

**Ventajas**
* Datasets montados nativamente (sin bugs de disco)
* Más I/O estable
* Entorno reproducible
* Ideal para EDA y pipelines de datos

## **1. 📚 Librerías y configuración**
---

### Instalar las librerías necesarias para leer datos y construir el dataset

In [1]:
!pip install kagglehub pyarrow requests

### Consolidar todas las librerías en una sola sección para evitar imports repetidos

In [3]:
import pandas as pd
import numpy as np
import kagglehub
import pyarrow as pa
import pyarrow.parquet as pq
import os
import requests
import time
import math
import fcntl  # En Windows reemplaza por msvcrt.locking si lo necesitas

## **2. ⬇️ Descargar el dataset presentado por Rafael para el analisis y confeción de uno mas completo**

---

### Extraer porciones de data por años y cargar por bloques

In [4]:
path = kagglehub.dataset_download("arvindnagaonkar/flight-delay")
print("Dataset descargado en:")
print(path)

# Lista los archivos disponibles dentro del dataset descargado
os.listdir(path)

# Construimos la ruta al archivo parquet
# ParquetFile permite leer el dataset sin cargarlo entero en memoria
PARQUET_FILE = os.path.join(path, "Flight_Delay.parquet")

OUT_FILE = "df_sample_equal_year.parquet"

BATCH_SIZE = 200_000
YEAR_COL = "Year"

# -------- 1) Primer pase: contar filas por Year (streaming) --------
pf = pq.ParquetFile(PARQUET_FILE)
counts = {}

for batch in pf.iter_batches(batch_size=BATCH_SIZE, columns=[YEAR_COL]):
    years = batch.column(0).to_numpy()
    uniq, cnt = np.unique(years, return_counts=True)
    for y, c in zip(uniq, cnt):
        counts[int(y)] = counts.get(int(y), 0) + int(c)

min_per_year = min(counts.values())  # mismo tamaño por Year

# (opcional) si quieres una fracción del mínimo:
# SAMPLE_FRAC = 0.10
# min_per_year = int(min_per_year * SAMPLE_FRAC)

print("Cantidad de registros en año:", counts)
print("Registros por Año a guardar:", min_per_year)

# -------- 2) Segundo pase: construir salida con N filas por Year --------
pf = pq.ParquetFile(PARQUET_FILE)

kept = {y: 0 for y in counts.keys()}
writer = None

for batch in pf.iter_batches(batch_size=BATCH_SIZE):  # lee todas las columnas
    df_chunk = batch.to_pandas()

    # seleccionar filas manteniendo cupo por year
    parts = []
    for y, g in df_chunk.groupby(YEAR_COL, sort=False):
        y = int(y)
        remaining = min_per_year - kept.get(y, 0)
        if remaining <= 0:
            continue
        take = g.iloc[:remaining]  # determinístico (primeras filas)
        kept[y] = kept.get(y, 0) + len(take)
        parts.append(take)

    if not parts:
        # si no hay nada que guardar de este batch, seguir
        continue

    out_df = pd.concat(parts, ignore_index=True)

    out_table = pa.Table.from_pandas(out_df, preserve_index=False)

    if writer is None:
        writer = pq.ParquetWriter(OUT_FILE, out_table.schema)
    writer.write_table(out_table)

    # early stop: si ya completaste todos los años, corta
    if all(v >= min_per_year for v in kept.values()):
        break

if writer is not None:
    writer.close()

print("La generación del archivo se ha terminado. Archivo creado: ", OUT_FILE)
print("Cantidades de las muestras seleccionadas: ", kept)

Dataset descargado en:
/kaggle/input/flight-delay
Cantidad de registros en año: {2018: 6438556, 2019: 6707642, 2020: 4148123, 2021: 5234456, 2022: 5706055, 2023: 1897840}
Registros por Año a guardar: 1897840
La generación del archivo se ha terminado. Archivo creado:  df_sample_equal_year.parquet
Cantidades de las muestras seleccionadas:  {2018: 1897840, 2019: 1897840, 2020: 1897840, 2021: 1897840, 2022: 1897840, 2023: 1897840}


## **3. 🧾 Leer el archivo de muestras**

---

### Convertir las muestras en un `DataFrame`

In [5]:
print ('Espere. Estoy leendo el archivo...')
df = pd.read_parquet(OUT_FILE)

print("La lectura ha terminado.")

Espere. Estoy leendo el archivo...
La lectura ha terminado.


### Cambiar los nombres de las columnas a minúsculas

In [7]:
df.columns = df.columns.str.lower()

## **4. 👀 Inspección visual de los datos**

---

### Visualizar las primeras 5 filas

In [8]:
df.head()

,year,month,dayofmonth,flightdate,marketing_airline_network,origincityname,destcityname,crsdeptime,deptime,depdelay,...,crselapsedtime,actualelapsedtime,airtime,distance,distancegroup,carrierdelay,weatherdelay,nasdelay,securitydelay,lateaircraftdelay
0,2018,1,15,2018-01-15,UA,"Newark, NJ","Charleston, SC",1845,1928.0,43.0,...,143.0,141.0,102.0,628.0,3,41.0,0.0,0.0,0.0,0.0
1,2018,1,16,2018-01-16,UA,"Newark, NJ","Charleston, SC",1835,1956.0,81.0,...,143.0,131.0,108.0,628.0,3,67.0,0.0,0.0,0.0,2.0
2,2018,1,17,2018-01-17,UA,"Newark, NJ","Charleston, SC",1835,1836.0,1.0,...,143.0,138.0,113.0,628.0,3,0.0,0.0,0.0,0.0,0.0
3,2018,1,18,2018-01-18,UA,"Newark, NJ","Charleston, SC",1845,1844.0,-1.0,...,143.0,134.0,92.0,628.0,3,0.0,0.0,0.0,0.0,0.0
4,2018,1,20,2018-01-20,UA,"Newark, NJ","Charleston, SC",1835,1829.0,-6.0,...,143.0,117.0,89.0,628.0,3,0.0,0.0,0.0,0.0,0.0


### ¿Cuántas filas y columnas hay?

In [9]:
print(f"Cantidad de columnas y filas: {df.shape}")

Cantidad de columnas y filas: (11387040, 29)


### Qué tipo de datos contiene cada una: ¿son números, fechas, texto?

In [10]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11387040 entries, 0 to 11387039
Data columns (total 29 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   year                       int64  
 1   month                      int64  
 2   dayofmonth                 int64  
 3   flightdate                 object 
 4   marketing_airline_network  object 
 5   origincityname             object 
 6   destcityname               object 
 7   crsdeptime                 int64  
 8   deptime                    float64
 9   depdelay                   float64
 10  depdelayminutes            float64
 11  taxiout                    float64
 12  wheelsoff                  float64
 13  wheelson                   float64
 14  taxiin                     float64
 15  crsarrtime                 int64  
 16  arrtime                    float64
 17  arrdelay                   float64
 18  arrdelayminutes            float64
 19  crselapsedtime             float64
 20  

### Que distribución de data hay por cada año

In [11]:
df["year"].value_counts(normalize=True)

year
2018    0.166667
2019    0.166667
2020    0.166667
2021    0.166667
2022    0.166667
2023    0.166667
Name: proportion, dtype: float64

### Estadística básica: medias, medianas y desviaciones estándar para entender la distribución de los números

In [12]:
df.describe()

,year,month,dayofmonth,crsdeptime,deptime,depdelay,depdelayminutes,taxiout,wheelsoff,wheelson,...,crselapsedtime,actualelapsedtime,airtime,distance,distancegroup,carrierdelay,weatherdelay,nasdelay,securitydelay,lateaircraftdelay
count,1.138704e+07,1.138704e+07,1.138704e+07,1.138704e+07,1.138704e+07,1.138704e+07,1.138704e+07,1.138704e+07,1.138704e+07,1.138704e+07,...,1.138704e+07,1.138704e+07,1.138704e+07,1.138704e+07,1.138704e+07,1.138704e+07,1.138704e+07,1.138704e+07,1.138704e+07,1.138704e+07
mean,2.020500e+03,7.043439e+00,1.575966e+01,1.321977e+03,1.326603e+03,9.950408e+00,1.369059e+01,1.678697e+01,1.350283e+03,1.463278e+03,...,1.395519e+02,1.324512e+02,1.081906e+02,7.743345e+02,3.571761e+00,5.092951e+00,8.002110e-01,2.728657e+00,2.584412e-02,5.176241e+00
std,1.707825e+00,4.625220e+00,8.788867e+00,4.817687e+02,4.938183e+02,5.205214e+01,5.091288e+01,9.502332e+00,4.947307e+02,5.179606e+02,...,7.108328e+01,7.062528e+01,6.888038e+01,5.805945e+02,2.283863e+00,3.443722e+01,1.600639e+01,1.496646e+01,1.544805e+00,2.731208e+01
min,2.018000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-3.420000e+02,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,...,-2.920000e+02,1.100000e+01,-1.412000e+03,1.600000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.019000e+03,1.000000e+00,8.000000e+00,9.150000e+02,9.190000e+02,-6.000000e+00,0.000000e+00,1.100000e+01,9.340000e+02,1.053000e+03,...,8.900000e+01,8.100000e+01,5.800000e+01,3.510000e+02,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.020500e+03,1.000000e+01,1.600000e+01,1.315000e+03,1.321000e+03,-3.000000e+00,0.000000e+00,1.400000e+01,1.335000e+03,1.500000e+03,...,1.210000e+02,1.150000e+02,9.000000e+01,6.180000e+02,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.022000e+03,1.100000e+01,2.300000e+01,1.725000e+03,1.733000e+03,3.000000e+00,3.000000e+00,1.900000e+01,1.747000e+03,1.903000e+03,...,1.700000e+02,1.620000e+02,1.370000e+02,1.011000e+03,5.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.023000e+03,1.200000e+01,3.100000e+01,2.359000e+03,2.400000e+03,4.413000e+03,4.413000e+03,1.439000e+03,2.400000e+03,2.400000e+03,...,1.645000e+03,7.640000e+02,7.270000e+02,5.095000e+03,1.100000e+01,4.218000e+03,2.098000e+03,1.660000e+03,1.460000e+03,2.962000e+03


## **5. 🧩 Creación de nuevas variables (Feature Engineering)**

---

### Se crea la variable temporal `hour`, la variable objetivo `dalayed` si el retraso en salida esta igual o mayór de 15 minutos

In [13]:
# Crear variable temporal 'hour' a partir de la hora programada
df["hour"] = df.select_dtypes(include='number')["crsdeptime"] // 100  # solo la hora

# Variable objetivo 'delayed': 1 si el retraso en salida >= 15 min
df["delayed"] = (df.select_dtypes(include='number')["depdelay"] >= 15).astype(int)

# Probabilidad de delay en la muestra
delay_rate = df["delayed"].mean()
print(f"Tasa de retraso: {delay_rate:.4f}")

# Ver valores únicos de la variable 'delayed'
unique_values = df["delayed"].unique()
print("Valores únicos en 'delayed':", unique_values)

Tasa de retraso: 0.1698
Valores únicos en 'delayed': [1 0]


### Revisamos la probilidad de retraso en diferentes horas

In [14]:
# Agrupar por hora y calcular la probabilidad de retraso
hour_delay = (
    df.groupby("hour")["delayed"]
    .mean()
    .sort_index()
)

# Mostrar la tabla de probabilidades por hora
print(hour_delay)

hour_std = hour_delay.std()
print(f"\nLa probabilidad de retraso de los vuelos en promedio\na lo largo de las diferentes horas del día: {hour_std:.2%}")

hour
0     0.180245
1     0.176570
2     0.239925
3     0.256506
4     0.229794
5     0.071609
6     0.074890
7     0.092360
8     0.110538
9     0.128383
10    0.145465
11    0.154597
12    0.168808
13    0.183205
14    0.193270
15    0.201940
16    0.213464
17    0.215501
18    0.230330
19    0.242814
20    0.233717
21    0.242783
22    0.213937
23    0.197289
Name: delayed, dtype: float64

La probabilidad de retraso de los vuelos en promedio
a lo largo de las diferentes horas del día: 5.49%


### Crear bins de distancia

In [15]:
# Crear bins de distancia (5 quintiles)
df["distance_bin"] = pd.qcut(df.select_dtypes(include='number')["distance"], q=5)

# Calcular probabilidad de retraso por rango de distancia
distance_delay = (
    df.groupby("distance_bin", observed=True)["delayed"]
    .mean()
)

print(distance_delay)

distance_bin
(15.999, 305.0]     0.156193
(305.0, 507.0]      0.156351
(507.0, 762.0]      0.167379
(762.0, 1107.0]     0.184747
(1107.0, 5095.0]    0.184449
Name: delayed, dtype: float64


### Crear columna con dia de la semana del vuelo

In [16]:
# Convertir a datetime (ya hecho en df, pero aseguramos si df_numeric no lo tiene)
# Asumimos que df_numeric y df tienen el mismo índice y número de filas
# y que df['flightdate'] ya es datetime de un paso anterior
df["day_of_week"] = pd.to_datetime(df["flightdate"]).dt.dayofweek

print("Distribución de vuelos puntuales y atrasados:")
df["delayed"].value_counts(normalize=True)

Distribución de vuelos puntuales y atrasados:


delayed
0    0.83024
1    0.16976
Name: proportion, dtype: float64

### Extraer código de estado `state_clean` y de la `state_clean` ciudad a partir de `origincityname` para una fusión con las ubicaciones de los aeropuertos

In [17]:
# "Dallas, TX" -> "Dallas"
# Necesario para el geocoding

tmp = (
    df["origincityname"]
    .astype(str)
    .str.split(",", n=1, expand=True)
)

df["city_clean"] = tmp[0].str.strip()
df["state_clean"] = tmp[1].str.strip()  # lo que va después de la coma

df[["origincityname", "city_clean", "state_clean"]].head()

,origincityname,city_clean,state_clean
0,"Newark, NJ",Newark,NJ
1,"Newark, NJ",Newark,NJ
2,"Newark, NJ",Newark,NJ
3,"Newark, NJ",Newark,NJ
4,"Newark, NJ",Newark,NJ


### Descargar y revisar datos con ubicación de los aeropuertos

In [18]:
URL_AIRPORTS = "https://davidmegginson.github.io/ourairports-data/airports.csv"

# 1) Descargar desde la URL (queda en memoria como DataFrame)
df_airports = pd.read_csv(URL_AIRPORTS)  # pandas permite leer CSV directo desde URL [web:497]

df_airports.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,icao_code,iata_code,gps_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total RF Heliport,40.070985,-74.933689,11.0,NaN,US,US-PA,Bensalem,no,NaN,NaN,K00A,00A,https://www.penndot.pa.gov/TravelInPA/airports...,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,NaN,NaN,00AA,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.947733,-151.692524,450.0,NaN,US,US-AK,Anchor Point,no,NaN,NaN,00AK,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,NaN,NaN,00AL,00AL,NaN,NaN,NaN
4,506791,00AN,small_airport,Katmai Lodge Airport,59.093287,-156.456699,80.0,NaN,US,US-AK,King Salmon,no,NaN,NaN,00AN,00AN,NaN,NaN,NaN


In [19]:
df_airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84389 entries, 0 to 84388
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 84389 non-null  int64  
 1   ident              84389 non-null  object 
 2   type               84389 non-null  object 
 3   name               84389 non-null  object 
 4   latitude_deg       84389 non-null  float64
 5   longitude_deg      84389 non-null  float64
 6   elevation_ft       69776 non-null  float64
 7   continent          44938 non-null  object 
 8   iso_country        84097 non-null  object 
 9   iso_region         84389 non-null  object 
 10  municipality       79621 non-null  object 
 11  scheduled_service  84389 non-null  object 
 12  icao_code          9535 non-null   object 
 13  iata_code          9060 non-null   object 
 14  gps_code           43811 non-null  object 
 15  local_code         35856 non-null  object 
 16  home_link          443

## **6. 🧹 Limpieza y preparación de datos**

---

### Dejar solo los aeropuertos de aviacion civil con vuelos programados

In [20]:
# 3) (Opcional) Filtrar para aviación civil "normal" (excluye heliports, seaplane)
allowed_types = {"small_airport", "medium_airport", "large_airport"}

df_airports = df_airports[
    df_airports["type"].isin(allowed_types) &
    df_airports["scheduled_service"].eq("yes")
].copy()

df_airports.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,icao_code,iata_code,gps_code,local_code,home_link,wikipedia_link,keywords
204,4650,03N,small_airport,Utirik Airport,11.222219,169.851429,4.0,OC,MH,MH-UTI,Utirik Island,yes,NaN,UTK,03N,03N,NaN,https://en.wikipedia.org/wiki/Utirik_Airport,NaN
290,6807,05AK,small_airport,Wasilla Creek Airpark,61.668301,-149.188004,620.0,NaN,US,US-AK,Palmer,yes,NaN,NaN,05AK,05AK,NaN,NaN,NaN
1387,7977,16A,small_airport,Nunapitchuk Airport,60.905591,-162.440454,12.0,NaN,US,US-AK,Nunapitchuk,yes,NaN,NUP,PPIT,16A,NaN,https://en.wikipedia.org/wiki/Nunapitchuk_Airport,NaN
2000,8638,1RL,small_airport,Point Roberts Airpark,48.977316,-123.079294,10.0,NaN,US,US-WA,Point Roberts,yes,NaN,NaN,K1RL,1RL,NaN,https://en.wikipedia.org/wiki/Point_Roberts_Ai...,NaN
3160,9819,2U7,small_airport,Stanley Airport,44.208500,-114.934998,6403.0,NaN,US,US-ID,Stanley,yes,NaN,NaN,NaN,2U7,NaN,NaN,NaN


### Buscar la información de unos aeropuertos por nombres

In [21]:
pattern = r"Odessa|Sheremet|Murmansk|Domodedovo International Airport|Murmansk Airport"

airport = df_airports.loc[
    df_airports["name"].astype(str).str.contains(pattern, case=False, na=False, regex=True)
]

airport

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,icao_code,iata_code,gps_code,local_code,home_link,wikipedia_link,keywords
65645,6487,UKOO,medium_airport,Odessa International Airport,46.426800,30.676500,172.0,EU,UA,UA-51,Odessa,yes,UKOO,ODS,UKOO,NaN,http://www.airport.od.ua/,https://en.wikipedia.org/wiki/Odessa_Internati...,"Odesa Central, Міжнародний аеропорт Одеса"
65676,6490,ULMM,medium_airport,Murmansk Airport,68.781700,32.750801,266.0,EU,RU,RU-MUR,Murmansk,yes,ULMM,MMK,ULMM,NaN,http://www.airport-murmansk.ru/,https://en.wikipedia.org/wiki/Murmansk_Airport,NaN
78844,26394,UUDD,large_airport,Domodedovo International Airport,55.408798,37.906300,588.0,EU,RU,RU-MOS,Moscow,yes,UUDD,DME,UUDD,NaN,http://www.domodedovo.ru/en/,https://en.wikipedia.org/wiki/Domodedovo_Inter...,"MOW, Аэропорт Домоде́дово"
78846,26396,UUEE,large_airport,Sheremetyevo International Airport,55.972599,37.414600,622.0,EU,RU,RU-MOS,Moscow,yes,UUEE,SVO,UUEE,NaN,http://svo.aero/en/,https://en.wikipedia.org/wiki/Sheremetyevo_Int...,"MOW, Международный аэропорт Шереметьево, svo, ..."


### Revisar si la información de los aeropuertos tiene los valores faltantes

- Si iata_code esta nulo, lo reemplazamos por un valor no nulo extraido de una de las siguentes columnas `icao_code`, `gps_code` o `local_code`

In [22]:
df_airports["airport_code"] = (
    df_airports["iata_code"]
      .fillna(df_airports["icao_code"])
      .fillna(df_airports["gps_code"])
      .fillna(df_airports["local_code"])
)

df_airports["airport_code"].isna().mean()

np.float64(0.007403869118700741)

### Muestra data con nulos

In [23]:
mask = df_airports["airport_code"].isna()

# inspección rápida
df_airports.loc[mask, ["ident","type","name","iso_country","scheduled_service",
                          "iata_code","icao_code","gps_code","local_code"]].head(20)

,ident,type,name,iso_country,scheduled_service,iata_code,icao_code,gps_code,local_code
9711,AF-0017,small_airport,Salerno South Airport,AF,yes,NaN,NaN,NaN,NaN
10578,AR-0503,small_airport,Gan Gan Field,AR,yes,NaN,NaN,NaN,NaN
10841,AR-0767,medium_airport,Aeroestación Yabotí,AR,yes,NaN,NaN,NaN,NaN
12904,BJ-0001,large_airport,Tourou International Airport,BJ,yes,NaN,NaN,NaN,NaN
22510,DE-0921,small_airport,Altstetten Ultralight Field,DE,yes,NaN,NaN,NaN,NaN
30996,ID-0004,small_airport,Okpahik Airstrip,ID,yes,NaN,NaN,NaN,NaN
31021,ID-0031,small_airport,Diphikin Airstrip,ID,yes,NaN,NaN,NaN,NaN
31084,ID-0094,small_airport,Long Rungan,ID,yes,NaN,NaN,NaN,NaN
31182,ID-0193,small_airport,Kubibkop Airport,ID,yes,NaN,NaN,NaN,NaN
31184,ID-0195,small_airport,Molding Airstrip,ID,yes,NaN,NaN,NaN,NaN


### Extraer solo los aeropuertos con valores no nulos en el `airport_code`

In [24]:
df_airports = df_airports.loc[~mask].copy()

In [25]:
df_airports.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4156 entries, 204 to 84383
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 4156 non-null   int64  
 1   ident              4156 non-null   object 
 2   type               4156 non-null   object 
 3   name               4156 non-null   object 
 4   latitude_deg       4156 non-null   float64
 5   longitude_deg      4156 non-null   float64
 6   elevation_ft       4078 non-null   float64
 7   continent          3003 non-null   object 
 8   iso_country        4147 non-null   object 
 9   iso_region         4156 non-null   object 
 10  municipality       4081 non-null   object 
 11  scheduled_service  4156 non-null   object 
 12  icao_code          3854 non-null   object 
 13  iata_code          4014 non-null   object 
 14  gps_code           4112 non-null   object 
 15  local_code         1107 non-null   object 
 16  home_link          1178 no

In [26]:
df_airports["airport_code"].isna().sum()

np.int64(0)

In [27]:
df_airports["airport_code"].sample(10)

84016    ENH
46076    MZO
83968    TGO
40535    PVD
63196    LRV
42715    PDV
84163    CZX
43453    BUD
80610    DJJ
7766     SSW
Name: airport_code, dtype: object

### Filtrar los aeropuertos de los EEUU

In [29]:
# airports: DataFrame ya cargado desde airports.csv (OurAirports)
# airports = pd.read_csv(URL_AIRPORTS)

# 1) Filtrar solo filas donde iso_region empieza con "US-"
df_us = df_airports.loc[
    df_airports["iso_region"].astype(str).str.startswith("US-"),
].copy()  # str.startswith para filtrar prefijos [web:526]

# 2) Crear state_clean = parte después del guion (US-AL -> AL)
df_us["state_clean"] = (
    df_us["iso_region"]
      .astype(str)
      .str.split("-", n=1, expand=True)[1]
      .str.strip()
)  # split con expand para crear columnas [web:412][web:405]

# 3) city_clean = municipality (solo renombre/copia)
df_us["city_clean"] = df_us["municipality"].astype(str)
df_us["latitude"] = df_us["latitude_deg"]
df_us["longitude"] = df_us["longitude_deg"]

# 4) Seleccionar columnas finales
df_export = df_us[[
    "type", "name", "latitude", "longitude",
    "state_clean", "city_clean", "airport_code"
]].copy()


### Revisar la muestra de cóodigos de aeropuertos

In [30]:
df_export.head()

,type,name,latitude,longitude,state_clean,city_clean,airport_code
290,small_airport,Wasilla Creek Airpark,61.668301,-149.188004,AK,Palmer,05AK
1387,small_airport,Nunapitchuk Airport,60.905591,-162.440454,AK,Nunapitchuk,NUP
2000,small_airport,Point Roberts Airpark,48.977316,-123.079294,WA,Point Roberts,K1RL
3160,small_airport,Stanley Airport,44.208500,-114.934998,ID,Stanley,2U7
3194,small_airport,Stuart Island West Airport,48.683991,-123.222849,WA,Friday Harbor,2WA3


### Guardar aeropuertos limpiados en un archivo CSV

In [31]:
# 5) Exportar a CSV
df_export.to_csv("airports_us_clean.csv", index=False)

## **7. 🛬 Cargar los aeropuertos depurados en un DataFrame y validarlos**

---

### Leer el archivo de aeropuertos

In [32]:
df_airports = pd.read_csv("airports_us_clean.csv")
df_airports.head()

,type,name,latitude,longitude,state_clean,city_clean,airport_code
0,small_airport,Wasilla Creek Airpark,61.668301,-149.188004,AK,Palmer,05AK
1,small_airport,Nunapitchuk Airport,60.905591,-162.440454,AK,Nunapitchuk,NUP
2,small_airport,Point Roberts Airpark,48.977316,-123.079294,WA,Point Roberts,K1RL
3,small_airport,Stanley Airport,44.208500,-114.934998,ID,Stanley,2U7
4,small_airport,Stuart Island West Airport,48.683991,-123.222849,WA,Friday Harbor,2WA3


### Unir el dataset de vuelos con las ubicaciones de los aeropuertos

In [33]:
#merge coordenadas y dataset

df = df.merge(df_airports, on=["city_clean", "state_clean"], how="left")
df.head()

,year,month,dayofmonth,flightdate,marketing_airline_network,origincityname,destcityname,crsdeptime,deptime,depdelay,...,delayed,distance_bin,day_of_week,city_clean,state_clean,type,name,latitude,longitude,airport_code
0,2018,1,15,2018-01-15,UA,"Newark, NJ","Charleston, SC",1845,1928.0,43.0,...,1,"(507.0, 762.0]",0,Newark,NJ,large_airport,Newark Liberty International Airport,40.692501,-74.168701,EWR
1,2018,1,16,2018-01-16,UA,"Newark, NJ","Charleston, SC",1835,1956.0,81.0,...,1,"(507.0, 762.0]",1,Newark,NJ,large_airport,Newark Liberty International Airport,40.692501,-74.168701,EWR
2,2018,1,17,2018-01-17,UA,"Newark, NJ","Charleston, SC",1835,1836.0,1.0,...,0,"(507.0, 762.0]",2,Newark,NJ,large_airport,Newark Liberty International Airport,40.692501,-74.168701,EWR
3,2018,1,18,2018-01-18,UA,"Newark, NJ","Charleston, SC",1845,1844.0,-1.0,...,0,"(507.0, 762.0]",3,Newark,NJ,large_airport,Newark Liberty International Airport,40.692501,-74.168701,EWR
4,2018,1,20,2018-01-20,UA,"Newark, NJ","Charleston, SC",1835,1829.0,-6.0,...,0,"(507.0, 762.0]",5,Newark,NJ,large_airport,Newark Liberty International Airport,40.692501,-74.168701,EWR


In [35]:
df.shape

(14534895, 40)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14534895 entries, 0 to 14534894
Data columns (total 40 columns):
 #   Column                     Dtype   
---  ------                     -----   
 0   year                       int64   
 1   month                      int64   
 2   dayofmonth                 int64   
 3   flightdate                 object  
 4   marketing_airline_network  object  
 5   origincityname             object  
 6   destcityname               object  
 7   crsdeptime                 int64   
 8   deptime                    float64 
 9   depdelay                   float64 
 10  depdelayminutes            float64 
 11  taxiout                    float64 
 12  wheelsoff                  float64 
 13  wheelson                   float64 
 14  taxiin                     float64 
 15  crsarrtime                 int64   
 16  arrtime                    float64 
 17  arrdelay                   float64 
 18  arrdelayminutes            float64 
 19  crselapsedtime     

### Eliminar valores nulos en los siguientes campos: `type`, `name`, `latitude`, `longitude`

In [37]:
cols_req = ["type", "name", "latitude", "longitude"]

print("Antes:", df.shape)
df_base = df.dropna(subset=cols_req, how="any").reset_index(drop=True)
print("Después:", df_base.shape)

Antes: (14534895, 40)
Después: (13426813, 40)


### Tomamar una fracción del dataset por cada año, porque consultar la API del clima es costoso en tiempo de ejecución

In [38]:
# Subsample MVP
N = 20_000
frac = N / len(df)

df_base = (
    df.groupby("year", group_keys=False)
      .sample(frac=frac, random_state=42)
)

df_base.head()


,year,month,dayofmonth,flightdate,marketing_airline_network,origincityname,destcityname,crsdeptime,deptime,depdelay,...,delayed,distance_bin,day_of_week,city_clean,state_clean,type,name,latitude,longitude,airport_code
399552,2018,1,30,2018-01-30,WN,"Denver, CO","Oakland, CA",2200,2155.0,-5.0,...,0,"(762.0, 1107.0]",1,Denver,CO,medium_airport,Rocky Mountain Metropolitan Airport,39.908798,-105.116997,BJC
137690,2018,1,25,2018-01-25,AA,"Denver, CO","Dallas/Fort Worth, TX",1935,1930.0,-5.0,...,0,"(507.0, 762.0]",3,Denver,CO,medium_airport,Rocky Mountain Metropolitan Airport,39.908798,-105.116997,BJC
364181,2018,1,11,2018-01-11,UA,"Houston, TX","Santa Ana, CA",1830,2149.0,199.0,...,1,"(1107.0, 5095.0]",3,Houston,TX,large_airport,George Bush Intercontinental Houston Airport,29.984400,-95.341400,IAH
575253,2018,1,21,2018-01-21,AA,"Dallas/Fort Worth, TX","Seattle, WA",1215,1212.0,-3.0,...,0,"(1107.0, 5095.0]",6,Dallas/Fort Worth,TX,NaN,NaN,NaN,NaN,NaN
1031130,2018,10,27,2018-10-27,AA,"Dallas/Fort Worth, TX","Memphis, TN",1700,1655.0,-5.0,...,0,"(305.0, 507.0]",5,Dallas/Fort Worth,TX,NaN,NaN,NaN,NaN,NaN


### Guardamos esa fracción en un archivo CSV

In [39]:
# 5) Exportar a CSV
df_base.to_csv("df_base_merged_sample_and_airports.csv", index=False)

## **8. 🌦️ Consultar la API externa de clima histórico y generar un dataset de entrenamiento enriquecido**

---

### Enriquecer el dataset de entrenamiento con información climática

Guardamos los resultados en bloques de 500 filas dentro del archivo `openmeteo_daily_cache.parquet`.

Como este archivo queda en el almacenamiento temporal del notebook, es recomendable descargarlo periódicamente para evitar pérdidas si la ejecución se interrumpe y así poder retomar el proceso sin repetir consultas a la API del clima.

Para continuar, vuelve a subir `openmeteo_daily_cache.parquet` al entorno temporal del notebook y ejecuta nuevamente las celdas siguientes, de modo que el proceso siga escribiendo sobre la caché existente.

La constante `HARD_CAP = 5_000` indica el número máximo total de consultas permitidas a la API del clima; así evitas ejecuciones muy largas o consumir demasiada cuota.

Asigna el valor según tus necesidades (por ejemplo, `HARD_CAP = 5_000` o `HARD_CAP = 15_000`) y ajusta en base al tiempo/costo que estás dispuesto a asumir; una buena práctica es partir conservador y luego aumentar si hace falta.

In [41]:
# código completo incorporando selección balanceada por año para keys_missing,
# de modo que cuando pidas max_api_calls=15000 no se vaya todo al primer año (p. ej. 2018),
# sino que tome una cantidad similar por año (y si un año tiene pocas claves faltantes, toma todas y redistribuye el resto).
#  La parte clave es usar groupby(...).apply(lambda g: g.sample(...)) para muestrear por grupo/año.

BASE_URL = "https://archive-api.open-meteo.com/v1/archive"
HARD_CAP = 20_000          # límite duro (tu tope global) — respeta tu configuración
DEFAULT_TIMEOUT = 15
MAX_RETRIES = 3
BACKOFF_BASE = 0.75        # segundos
WRITE_BATCH_SIZE = 500     # cada cuántas respuestas persistimos


def _safe_sleep(last_call_ts, min_interval_s):
    now = time.time()
    wait = (last_call_ts + min_interval_s) - now
    if wait > 0:
        time.sleep(wait)
    return time.time()


def _retry_get(session, url, params, timeout=DEFAULT_TIMEOUT, max_retries=MAX_RETRIES):
    for attempt in range(1, max_retries + 1):
        try:
            r = session.get(url, params=params, timeout=timeout)
            r.raise_for_status()
            return r
        except requests.RequestException:
            if attempt == max_retries:
                raise
            back = BACKOFF_BASE * (2 ** (attempt - 1))
            time.sleep(back)
    raise RuntimeError("Exhausted retries unexpectedly")


def fetch_daily_weather_for_point_date(lat, lon, date_str, timezone="UTC", session=None, timeout=DEFAULT_TIMEOUT):
    params = {
        "latitude": float(lat),
        "longitude": float(lon),
        "start_date": date_str,
        "end_date": date_str,
        "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max",
        "timezone": timezone,
    }
    s = session or requests.Session()
    r = _retry_get(s, BASE_URL, params=params, timeout=timeout, max_retries=MAX_RETRIES)
    data = r.json()

    d = data.get("daily", {})
    row = {
        "flightdate": date_str,
        "temp_max": d.get("temperature_2m_max", [None])[0],
        "temp_min": d.get("temperature_2m_min", [None])[0],
        "precipitation_sum": d.get("precipitation_sum", [None])[0],
        "wind_speed_max": d.get("windspeed_10m_max", [None])[0],
    }
    if row["temp_max"] is not None and row["temp_min"] is not None:
        row["temp_mean"] = (float(row["temp_max"]) + float(row["temp_min"])) / 2.0
    else:
        row["temp_mean"] = None
    return row


def _ensure_dir(path):
    d = os.path.dirname(os.path.abspath(path))
    if d and not os.path.exists(d):
        os.makedirs(d, exist_ok=True)


def _lock_file(path, mode="a+b"):
    f = open(path, mode)
    try:
        fcntl.flock(f.fileno(), fcntl.LOCK_EX)
    except Exception:
        f.close()
        raise
    return f


def _write_parquet_atomic(df: pd.DataFrame, path: str, tmp_suffix=".tmp"):
    tmp_path = f"{path}{tmp_suffix}"
    df.to_parquet(tmp_path, index=False)
    os.replace(tmp_path, path)


def _load_cache(cache_path: str) -> pd.DataFrame:
    cols = ["_lat_r","_lon_r","flightdate","temp_max","temp_min","temp_mean","precipitation_sum","wind_speed_max"]
    if os.path.exists(cache_path):
        try:
            df_cache = pd.read_parquet(cache_path)
            for c in cols:
                if c not in df_cache.columns:
                    df_cache[c] = pd.Series(dtype="float64" if c.startswith("temp") or c in ["precipitation_sum","wind_speed_max"] else "object")
            df_cache = df_cache.drop_duplicates(subset=["_lat_r", "_lon_r", "flightdate"], keep="last")
            return df_cache
        except Exception:
            corrupt = cache_path + ".corrupt"
            os.replace(cache_path, corrupt)
            print(f"[cache] Archivo corrupto movido a: {corrupt}. Se reinicia cache.")
    return pd.DataFrame(columns=cols)


def _balance_keys_by_year(keys_missing: pd.DataFrame, max_api_calls: int, random_state: int = 42, exclude_years=None) -> pd.DataFrame:
    """
    Devuelve un subconjunto de keys_missing, muestreado de forma balanceada por 'year'
    derivado desde flightdate. Usa groupby+sample. [web:696][web:712]
    """
    exclude_years = set(exclude_years or [])
    km = keys_missing.copy()
    km["year"] = pd.to_datetime(km["flightdate"], errors="coerce").dt.year

    if exclude_years:
        km = km[~km["year"].isin(exclude_years)].copy()

    # Si no hay year parseable, fallback a sample global
    years = sorted(km["year"].dropna().unique())
    if len(years) == 0:
        out = km.sample(n=min(len(km), max_api_calls), random_state=random_state)
        return out.drop(columns=["year"], errors="ignore")

    # Target por año
    per_year = math.ceil(max_api_calls / len(years))

    # Sample por grupo/año (sin reemplazo; si hay menos, toma todo)
    sampled = (
        km.sample(frac=1, random_state=random_state)
          .groupby("year", group_keys=False)
          .head(per_year)
    )

    # Si sobraron por ceil, recorta a max_api_calls; si faltaron, rellena con el resto
    if len(sampled) > max_api_calls:
        sampled = sampled.sample(n=max_api_calls, random_state=random_state)

    elif len(sampled) < max_api_calls:
        picked_keys = (
            sampled["_lat_r"].astype(str) + "|" +
            sampled["_lon_r"].astype(str) + "|" +
            sampled["flightdate"].astype(str)
        )
        picked_set = set(picked_keys.values)

        all_keys = (
            km["_lat_r"].astype(str) + "|" +
            km["_lon_r"].astype(str) + "|" +
            km["flightdate"].astype(str)
        )
        rest = km.loc[~all_keys.isin(picked_set)]
        need = max_api_calls - len(sampled)
        if len(rest) > 0 and need > 0:
            sampled = pd.concat(
                [sampled, rest.sample(n=min(len(rest), need), random_state=random_state)],
                ignore_index=True
            )

    return sampled.drop(columns=["year"], errors="ignore")


def enrich_df_base_with_weather_daily_cached(
    df_base: pd.DataFrame,
    timezone="UTC",
    max_requests_per_second=2.0,
    max_api_calls=20000,
    round_coords_decimals=3,
    cache_path="/content/drive/MyDrive/openmeteo_daily_cache.parquet",
    show_progress=True,
    random_state=42,
    exclude_years=None,        # ej: [2018] si quieres saltarte 2018
) -> pd.DataFrame:

    max_api_calls = min(int(max_api_calls), HARD_CAP)
    _ensure_dir(cache_path)

    # Normaliza fechas y claves
    df = df_base.copy()
    df["flightdate"] = pd.to_datetime(df["flightdate"]).dt.strftime("%Y-%m-%d")
    df["_lat_r"] = df["latitude"].round(round_coords_decimals)
    df["_lon_r"] = df["longitude"].round(round_coords_decimals)

    keys = (
        df.loc[df["_lat_r"].notna() & df["_lon_r"].notna(), ["_lat_r", "_lon_r", "flightdate"]]
          .drop_duplicates()
          .reset_index(drop=True)
    )

    # Carga cache
    df_cache = _load_cache(cache_path)

    # Anti-join vectorizado
    if not df_cache.empty:
        cache_key = (
            df_cache["_lat_r"].astype(str) + "|" +
            df_cache["_lon_r"].astype(str) + "|" +
            df_cache["flightdate"].astype(str)
        )
        cached_set = set(cache_key.values)
    else:
        cached_set = set()

    keys_key = keys["_lat_r"].astype(str) + "|" + keys["_lon_r"].astype(str) + "|" + keys["flightdate"].astype(str)
    missing_mask = ~keys_key.isin(cached_set)
    keys_missing = keys.loc[missing_mask].reset_index(drop=True)

    total_missing = len(keys_missing)
    if total_missing == 0:
        print(f"[ok] No hay claves faltantes. Cache: {len(df_cache)} filas. Archivo: {os.path.abspath(cache_path)}")

    # IMPORTANTE: balancea por año ANTES de aplicar el cap final
    if total_missing > 0:
        keys_missing = _balance_keys_by_year(
            keys_missing=keys_missing,
            max_api_calls=max_api_calls,
            random_state=random_state,
            exclude_years=exclude_years
        )

    to_fetch = len(keys_missing)
    print(f"[plan] Únicas totales: {len(keys)} | En cache: {len(cached_set)} | A consultar: {to_fetch} (cap máx: {HARD_CAP})")

    if to_fetch == 0:
        df_enriched = df.merge(
            df_cache.drop_duplicates(subset=["_lat_r","_lon_r","flightdate"], keep="last"),
            how="left",
            on=["_lat_r","_lon_r","flightdate"]
        )
        df_enriched.drop(columns=["_lat_r", "_lon_r"], inplace=True)
        return df_enriched

    # Rate limit
    min_interval_s = 1.0 / max_requests_per_second if max_requests_per_second > 0 else 0.0
    last_call_ts = 0.0
    session = requests.Session()

    # Barra de progreso opcional
    try:
        from tqdm import tqdm
        bar = tqdm(total=to_fetch, unit="req", disable=not show_progress)
    except Exception:
        bar = None
        print("[info] tqdm no disponible. Continuando sin barra de progreso.")

    new_rows = []
    written_since_last = 0
    processed = 0
    start_ts = time.time()

    def persist_incremental(df_cache_local, new_rows_batch):
        if not new_rows_batch:
            return df_cache_local
        #df_new_local = pd.DataFrame(new_rows_batch)
        #df_cache_local = pd.concat([df_cache_local, df_new_local], ignore_index=True)
        df_new_local = pd.DataFrame(new_rows_batch)
        if df_new_local.empty:
            return df_cache_local

        df_cache_local = pd.concat([df_cache_local, df_new_local], ignore_index=True)
        df_cache_local = df_cache_local.drop_duplicates(subset=["_lat_r","_lon_r","flightdate"], keep="last")
        with _lock_file(cache_path, mode="a+b"):
            _write_parquet_atomic(df_cache_local, cache_path)
        return df_cache_local

    for _, k in keys_missing.iterrows():
        if processed >= HARD_CAP:
            break

        last_call_ts = _safe_sleep(last_call_ts, min_interval_s)

        try:
            row = fetch_daily_weather_for_point_date(
                lat=k["_lat_r"], lon=k["_lon_r"], date_str=k["flightdate"],
                timezone=timezone, session=session
            )
            row["_lat_r"] = k["_lat_r"]
            row["_lon_r"] = k["_lon_r"]
        except requests.RequestException:
            row = {
                "_lat_r": k["_lat_r"],
                "_lon_r": k["_lon_r"],
                "flightdate": k["flightdate"],
                "temp_max": None,
                "temp_min": None,
                "temp_mean": None,
                "precipitation_sum": None,
                "wind_speed_max": None,
            }

        new_rows.append(row)
        processed += 1
        written_since_last += 1

        if bar:
            bar.update(1)
            elapsed = max(time.time() - start_ts, 1e-6)
            rate = processed / elapsed
            remaining = to_fetch - processed
            eta = remaining / rate if rate > 0 else float("inf")
            bar.set_postfix({"rate": f"{rate:.2f}/s", "ETA": f"{eta/60:.1f}m"})

        if written_since_last >= WRITE_BATCH_SIZE:
            df_cache = persist_incremental(df_cache, new_rows)
            print(f"[persist] Escrito batch de {written_since_last}. Total cache: {len(df_cache)}")
            new_rows.clear()
            written_since_last = 0

    if new_rows:
        df_cache = persist_incremental(df_cache, new_rows)
        print(f"[persist] Escrito batch final de {len(new_rows)}. Total cache: {len(df_cache)}")
        new_rows.clear()

    if bar:
        bar.close()

    # Merge final
    df_enriched = df.merge(
        df_cache.drop_duplicates(subset=["_lat_r","_lon_r","flightdate"], keep="last"),
        how="left",
        on=["_lat_r","_lon_r","flightdate"]
    )
    df_enriched.drop(columns=["_lat_r", "_lon_r"], inplace=True)

    print(f"[done] Enriquecidas {len(df_enriched)} filas. Cache en: {os.path.abspath(cache_path)}")
    return df_enriched

### Ejecutar la función que consulta la API y actualiza el archivo de caché

In [42]:
df_base = enrich_df_base_with_weather_daily_cached(
    df_base,
    timezone="auto",
    max_requests_per_second=2.0,
    max_api_calls=15000,
    round_coords_decimals=3,
    cache_path="openmeteo_daily_cache.parquet",
    #exclude_years=[2018, 2019], # opcional. Si ya tienes info de 2018 y 2019 y quieres priorizar otros años
    random_state=42
)

[plan] Únicas totales: 14742 | En cache: 0 | A consultar: 14742 (cap máx: 20000)


  3%|▎         | 500/14742 [04:09<1:58:39,  2.00req/s, rate=2.00/s, ETA=118.5m]/tmp/ipykernel_55/3847445582.py:256: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_cache_local = pd.concat([df_cache_local, df_new_local], ignore_index=True)


[persist] Escrito batch de 500. Total cache: 500


  7%|▋         | 1000/14742 [08:20<1:54:51,  1.99req/s, rate=2.00/s, ETA=114.5m]

[persist] Escrito batch de 500. Total cache: 1000


...

[persist] Escrito batch de 500. Total cache: 14000


 98%|█████████▊| 14500/14742 [2:30:46<02:01,  2.00req/s, rate=1.60/s, ETA=2.5m]

[persist] Escrito batch de 500. Total cache: 14500


100%|██████████| 14742/14742 [2:32:48<00:00,  1.61req/s, rate=1.61/s, ETA=0.0m]


[persist] Escrito batch final de 242. Total cache: 14742
[done] Enriquecidas 20001 filas. Cache en: /kaggle/working/openmeteo_daily_cache.parquet


### Exportar el `DataFrame` a CSV: `dataset_con_meteo_out.csv`

In [43]:
df_base.to_csv("dataset_con_meteo_out.csv", index=False, encoding="utf-8")

### Cargar el CSV en un nuevo `DataFrame`

In [45]:
df = pd.read_csv("dataset_con_meteo_out.csv")

### Inspeccionar `DataFrame`


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20001 entries, 0 to 20000
Data columns (total 45 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   year                       20001 non-null  int64  
 1   month                      20001 non-null  int64  
 2   dayofmonth                 20001 non-null  int64  
 3   flightdate                 20001 non-null  object 
 4   marketing_airline_network  20001 non-null  object 
 5   origincityname             20001 non-null  object 
 6   destcityname               20001 non-null  object 
 7   crsdeptime                 20001 non-null  int64  
 8   deptime                    20001 non-null  float64
 9   depdelay                   20001 non-null  float64
 10  depdelayminutes            20001 non-null  float64
 11  taxiout                    20001 non-null  float64
 12  wheelsoff                  20001 non-null  float64
 13  wheelson                   20001 non-null  flo

In [47]:
df.head()

,year,month,dayofmonth,flightdate,marketing_airline_network,origincityname,destcityname,crsdeptime,deptime,depdelay,...,type,name,latitude,longitude,airport_code,temp_max,temp_min,temp_mean,precipitation_sum,wind_speed_max
0,2018,1,30,2018-01-30,WN,"Denver, CO","Oakland, CA",2200,2155.0,-5.0,...,medium_airport,Rocky Mountain Metropolitan Airport,39.908798,-105.116997,BJC,15.9,-1.1,7.4,0.0,20.4
1,2018,1,25,2018-01-25,AA,"Denver, CO","Dallas/Fort Worth, TX",1935,1930.0,-5.0,...,medium_airport,Rocky Mountain Metropolitan Airport,39.908798,-105.116997,BJC,6.9,-9.9,-1.5,0.0,16.6
2,2018,1,11,2018-01-11,UA,"Houston, TX","Santa Ana, CA",1830,2149.0,199.0,...,large_airport,George Bush Intercontinental Houston Airport,29.984400,-95.341400,IAH,18.0,4.2,11.1,13.5,32.6
3,2018,1,21,2018-01-21,AA,"Dallas/Fort Worth, TX","Seattle, WA",1215,1212.0,-3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,10,27,2018-10-27,AA,"Dallas/Fort Worth, TX","Memphis, TN",1700,1655.0,-5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df.shape

(20001, 45)

### Renombrar las nuevas columnas agregadas previamente

In [50]:
df = df.rename(columns={
    "precipitation_sum": "precipitation",
    "wind_speed_max": "wind_speed",
})

### Revisar cuántos valores nulos tienen estas columnas

In [51]:
df[["temp_mean", "precipitation", "wind_speed"]].isna().sum()

temp_mean        2551
precipitation    2551
wind_speed       2551
dtype: int64

### Revisar cuántos valores nulos tiene el dataset, agrupado por año

In [52]:
# null por años
cols = ["temp_mean", "precipitation", "wind_speed"]

mask_any_na = df[cols].isna().any(axis=1)
na_rows_by_year = df.loc[mask_any_na].groupby("year").size().sort_index()

na_rows_by_year

year
2018    388
2019    423
2020    479
2021    431
2022    437
2023    393
dtype: int64

### Revisar cuántos valores no nulos tenemos por año

In [53]:
# No-null por año (para columnas específicas)

cols = ["temp_mean", "precipitation", "wind_speed"]

not_null_by_year = df.groupby("year")[cols].count().sort_index()
not_null_by_year

,temp_mean,precipitation,wind_speed
year,,,
2018,2944,2944,2944
2019,2900,2900,2900
2020,2831,2831,2831
2021,2905,2905,2905
2022,2914,2914,2914
2023,2956,2956,2956


### Revisar una muestra de los datos con valores nulos

In [54]:
df.loc[mask_any_na, ["year", "flightdate"] + cols].sample(20)

,year,flightdate,temp_mean,precipitation,wind_speed
9358,2020,2020-12-28,NaN,NaN,NaN
5267,2019,2019-12-09,NaN,NaN,NaN
19838,2023,2023-01-07,NaN,NaN,NaN
9637,2020,2020-11-13,NaN,NaN,NaN
13128,2021,2021-12-13,NaN,NaN,NaN
13286,2021,2021-12-29,NaN,NaN,NaN
2219,2018,2018-11-20,NaN,NaN,NaN
9457,2020,2020-01-29,NaN,NaN,NaN
8721,2020,2020-01-04,NaN,NaN,NaN
10203,2021,2021-12-05,NaN,NaN,NaN


### Eliminar todos los registros con nulos en las columnas nuevas y conservar solo los que tienen información meteorológica; revisar el resultado

In [55]:
# Eliminamos filas sin información climática

df_ml = df.dropna(
    subset=["temp_mean", "precipitation", "wind_speed"]
)

df_ml.shape

(17450, 45)

In [56]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17450 entries, 0 to 20000
Data columns (total 45 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   year                       17450 non-null  int64  
 1   month                      17450 non-null  int64  
 2   dayofmonth                 17450 non-null  int64  
 3   flightdate                 17450 non-null  object 
 4   marketing_airline_network  17450 non-null  object 
 5   origincityname             17450 non-null  object 
 6   destcityname               17450 non-null  object 
 7   crsdeptime                 17450 non-null  int64  
 8   deptime                    17450 non-null  float64
 9   depdelay                   17450 non-null  float64
 10  depdelayminutes            17450 non-null  float64
 11  taxiout                    17450 non-null  float64
 12  wheelsoff                  17450 non-null  float64
 13  wheelson                   17450 non-null  float64


### Eliminar las columnas sobrantes y revisar el resultado

In [57]:
df_ml = df_ml.drop(columns=["temp_min", "temp_max"])

In [58]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17450 entries, 0 to 20000
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   year                       17450 non-null  int64  
 1   month                      17450 non-null  int64  
 2   dayofmonth                 17450 non-null  int64  
 3   flightdate                 17450 non-null  object 
 4   marketing_airline_network  17450 non-null  object 
 5   origincityname             17450 non-null  object 
 6   destcityname               17450 non-null  object 
 7   crsdeptime                 17450 non-null  int64  
 8   deptime                    17450 non-null  float64
 9   depdelay                   17450 non-null  float64
 10  depdelayminutes            17450 non-null  float64
 11  taxiout                    17450 non-null  float64
 12  wheelsoff                  17450 non-null  float64
 13  wheelson                   17450 non-null  float64


### Revisar y eliminar registros duplicados

In [59]:
print("Buscando duplicados...")
num_duplicates = df_ml.duplicated().sum()
print(f"Número de filas duplicadas: {num_duplicates}")

Buscando duplicados...
Número de filas duplicadas: 0


In [60]:
print("Registros antes de eliminar duplicados:", len(df_ml))
df_ml = df_ml.drop_duplicates(keep="first").reset_index(drop=True)
print("Registros después de eliminar duplicados:", len(df_ml))

Registros antes de eliminar duplicados: 17450
Registros después de eliminar duplicados: 17450


### Asegurar que solo queden aeropuertos de aviación civil con vuelos programados

In [62]:
allowed_types = {"small_airport", "medium_airport", "large_airport"}
df_ml = df_ml[df_ml["type"].isin(allowed_types)].copy()

In [63]:
df_ml.loc[df_ml["type"].isin(allowed_types), "type"].unique()

array(['medium_airport', 'large_airport', 'small_airport'], dtype=object)

In [64]:
df_ml.sample(20)


,year,month,dayofmonth,flightdate,marketing_airline_network,origincityname,destcityname,crsdeptime,deptime,depdelay,...,city_clean,state_clean,type,name,latitude,longitude,airport_code,temp_mean,precipitation,wind_speed
15015,2023,1,24,2023-01-24,DL,"Tulsa, OK","Atlanta, GA",601,554.0,-7.0,...,Tulsa,OK,large_airport,Tulsa International Airport,36.198399,-95.888100,TUL,1.05,9.8,17.3
8137,2020,12,19,2020-12-19,UA,"Miami, FL","Houston, TX",1645,1647.0,2.0,...,Miami,FL,large_airport,Miami International Airport,25.793200,-80.290604,MIA,18.90,0.0,15.1
15738,2023,3,29,2023-03-29,AA,"Philadelphia, PA","Indianapolis, IN",1535,1530.0,-5.0,...,Philadelphia,PA,large_airport,Philadelphia International Airport,39.871899,-75.241096,PHL,7.25,0.0,17.6
6586,2020,10,19,2020-10-19,UA,"Orlando, FL","Newark, NJ",1842,1841.0,-1.0,...,Orlando,FL,large_airport,Orlando Sanford International Airport,28.777599,-81.237503,SFB,25.55,3.5,23.5
3560,2019,11,10,2019-11-10,DL,"Atlanta, GA","Baton Rouge, LA",1102,1059.0,-3.0,...,Atlanta,GA,medium_airport,DeKalb Peachtree Airport,33.876330,-84.302130,PDK,9.20,0.0,9.3
12632,2022,1,31,2022-01-31,DL,"Atlanta, GA","Las Vegas, NV",815,809.0,-6.0,...,Atlanta,GA,large_airport,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,ATL,6.10,0.0,10.8
16140,2023,1,15,2023-01-15,AA,"Missoula, MT","Dallas/Fort Worth, TX",1335,1323.0,-12.0,...,Missoula,MT,large_airport,Missoula International Airport,46.915816,-114.091107,MSO,1.65,2.3,7.9
2229,2018,10,25,2018-10-25,DL,"New York, NY","Louisville, KY",727,719.0,-8.0,...,New York,NY,large_airport,John F Kennedy International Airport,40.639447,-73.779317,JFK,7.50,0.0,21.4
3938,2019,1,19,2019-01-19,UA,"Cleveland, OH","Washington, DC",1020,1019.0,-1.0,...,Cleveland,OH,large_airport,Cleveland Hopkins International Airport,41.411701,-81.849800,CLE,-2.95,16.2,36.8
4085,2019,1,12,2019-01-12,AA,"Phoenix, AZ","Palm Springs, CA",2113,2103.0,-10.0,...,Phoenix,AZ,large_airport,Phoenix Sky Harbor International Airport,33.435302,-112.005905,PHX,13.25,1.6,19.5


In [65]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17450 entries, 0 to 17449
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   year                       17450 non-null  int64  
 1   month                      17450 non-null  int64  
 2   dayofmonth                 17450 non-null  int64  
 3   flightdate                 17450 non-null  object 
 4   marketing_airline_network  17450 non-null  object 
 5   origincityname             17450 non-null  object 
 6   destcityname               17450 non-null  object 
 7   crsdeptime                 17450 non-null  int64  
 8   deptime                    17450 non-null  float64
 9   depdelay                   17450 non-null  float64
 10  depdelayminutes            17450 non-null  float64
 11  taxiout                    17450 non-null  float64
 12  wheelsoff                  17450 non-null  float64
 13  wheelson                   17450 non-null  flo

## **9. ⬇️ Exportar el dataset final a CSV y forzar su descarga (solo en Colab)**

---

In [66]:
OUTPUT_CSV = "dataset_vuelos_clima_final.csv"
df_ml.to_csv(OUTPUT_CSV, index=False)

OUTPUT_CSV

# Si tienes recursos suficientes y estás ejecutando este notebook en Google Colab,
# puedes descomentar el bloque de abajo para forzar la descarga del archivo generado.
"""
from google.colab import files
files.download(OUTPUT_CSV)
"""

'\nfrom google.colab import files\nfiles.download(OUTPUT_CSV)\n'